# VS Code ↔ GitHub ↔ Colab 통합 워크플로 노트북
이 노트북은 로컬(VS Code)에서 작성/추가한 노트북을 GitHub에 푸시하고 Colab에서 재사용·실행·편집 후 다시 반영하는 전체 절차를 15개 섹션으로 정리합니다.


## 1. Git 설정 및 환경 확인
로컬/Colab 환경 차이 파악을 위해 Git과 Python 패키지 목록을 확인합니다.

```bash
!git --version
```
```bash
!pip list | head -n 25
```
Git 사용자 정보 (Colab에서 push 필요 시 설정):
```bash
!git config --global user.name "Your Name"
!git config --global user.email "you@example.com"
!git config --list | grep user.
```


## 2. Repo 초기화와 첫 커밋 (Notebook 포함)
로컬 VS Code에서 (이미 git이면 생략 가능):
```bash
git init
git add .
git commit -m "init notebook"
```
Colab 환경에서 클론 후 상태:
```bash
!git status
```


## 3. GitHub 원격 저장소 연결 및 푸시
```bash
git remote add origin https://github.com/OWNER/REPO.git
git branch -M main
git push -u origin main
```
토큰 403 발생 시:
```bash
git remote set-url origin https://TOKEN@github.com/OWNER/REPO.git
```


## 4. Colab에서 GitHub Notebook 열기
직접 URL 패턴:
```
https://colab.research.google.com/github/OWNER/REPO/blob/main/analysis/colab_pipeline.ipynb
```
또는 Colab > File > Open Notebook > GitHub 탭 활용.


# GCP + Colab 통합 워크플로 (RENDER 대체)

이 섹션은 RENDER에서 GCP로 전환했을 때의 코랩 통합 이점을 보여줍니다.

## GCP 이점:
1. **자동 동기화**: Colab → Google Drive → Cloud Storage
2. **API 통합**: 동일한 Google 계정으로 모든 서비스 접근
3. **무료 할당량**: Cloud Run 2M 요청, Storage 5GB, Firestore 1GB
4. **ML 워크플로**: Vertex AI Notebooks와 원활한 연동

In [ ]:
# GCP 연동 설정 (코랩에서 실행)
!pip install google-cloud-storage google-cloud-firestore

import json
import uuid
from datetime import datetime
from google.cloud import storage, firestore
import os

# 프로젝트 설정 (실제 값으로 교체)
PROJECT_ID = "your-techno-prompt-project"
BUCKET_NAME = "techno-prompt-storage"

# 서비스 계정 키 설정 (Colab Secrets에서 가져오기)
# from google.colab import userdata
# service_account_key = userdata.get('GCP_SERVICE_ACCOUNT')
# with open('/tmp/service-account.json', 'w') as f:
#     f.write(service_account_key)
# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/tmp/service-account.json'

def upload_analysis_result(analysis_type, input_data, results, processing_time_ms=0):
    """
    분석 결과를 GCP에 업로드하는 함수
    RENDER에서는 불가능한 직접적인 Google 서비스 통합
    """
    
    # 결과 구조화
    result = {
        "timestamp": datetime.now().isoformat(),
        "analysisType": analysis_type,
        "inputData": input_data,
        "results": results,
        "metadata": {
            "colabSessionId": str(uuid.uuid4())[:8],
            "processingTimeMs": processing_time_ms,
            "modelUsed": "colab-analysis",
            "source": "google-colab"
        }
    }
    
    try:
        # Cloud Storage 업로드
        client = storage.Client(project=PROJECT_ID)
        bucket = client.bucket(BUCKET_NAME)
        
        file_name = f"analysis/{analysis_type}/{result['timestamp']}-{result['metadata']['colabSessionId']}.json"
        blob = bucket.blob(file_name)
        blob.upload_from_string(
            json.dumps(result, indent=2),
            content_type='application/json'
        )
        
        # Firestore 메타데이터 저장 (빠른 조회용)
        db = firestore.Client(project=PROJECT_ID)
        db.collection('colab_analyses').document(
            f"{result['timestamp']}-{result['metadata']['colabSessionId']}"
        ).set({
            'fileName': file_name,
            'analysisType': analysis_type,
            'timestamp': result['timestamp'],
            'processingTimeMs': processing_time_ms,
            'colabSessionId': result['metadata']['colabSessionId'],
            'createdAt': firestore.SERVER_TIMESTAMP
        })
        
        print(f"✅ Analysis uploaded successfully: {file_name}")
        print(f"📊 Type: {analysis_type}, Session: {result['metadata']['colabSessionId']}")
        return file_name
        
    except Exception as e:
        print(f"❌ Upload failed: {str(e)}")
        return None

# 사용 예시
sample_result = upload_analysis_result(
    "audio-pattern",
    {"bpm": 128, "genre": "techno", "samples": ["kick.wav", "hihat.wav"]},
    {"detected_patterns": ["four-on-floor", "syncopated-hi-hats"], "confidence": 0.87},
    processing_time_ms=1500
)

print("GCP 통합 설정 완료! 🎉")

# 🎵 음악 분석 특화 워크플로

GCP에서만 가능한 음악 분석 파이프라인을 설정합니다.

## 분석 가능한 항목:
1. **BPM 분석**: 리듬 패턴 기반 템포 추출
2. **장르 분류**: Vertex AI AutoML 활용
3. **음악 구조**: 섹션별 분할 (Intro, Verse, Chorus 등)
4. **악기 탐지**: 사용된 악기 종류 분석
5. **화음 진행**: 코드 시퀀스 추출

In [ ]:
# 음악 분석 전용 함수들
import librosa
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
from IPython.display import Audio, display

def analyze_audio_file(file_path, upload_to_gcp=True):
    """
    음악 파일을 종합 분석하고 결과를 GCP에 업로드
    RENDER에서는 불가능한 GPU 가속 + 클라우드 저장소 통합
    """
    print(f"🎵 분석 시작: {file_path}")
    
    # 1. 오디오 로드 (librosa 활용)
    y, sr = librosa.load(file_path, duration=60)  # 처음 1분만 분석 (비용 절약)
    
    # 2. BPM 분석
    tempo, beats = librosa.beat.beat_track(y=y, sr=sr, units='time')
    print(f"🥁 BPM: {tempo:.1f}")
    
    # 3. 스펙트럼 분석 (주파수 특성)
    chroma = librosa.feature.chroma_stft(y=y, sr=sr)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    spectral_centroids = librosa.feature.spectral_centroid(y=y, sr=sr)[0]
    
    # 4. 장르 추정 (간단한 특성 기반)
    brightness = np.mean(spectral_centroids)
    rhythmic_regularity = np.std(np.diff(beats))
    
    if tempo > 130 and brightness > 2000:
        predicted_genre = "techno"
    elif tempo < 100 and rhythmic_regularity < 0.5:
        predicted_genre = "ambient"
    elif 110 <= tempo <= 130:
        predicted_genre = "house"
    else:
        predicted_genre = "electronic"
    
    print(f"🎼 예상 장르: {predicted_genre}")
    
    # 5. 음악 구조 분석 (섹션 분할)
    # 스펙트럼 변화점을 통한 구간 탐지
    chroma_stack = librosa.util.stack([chroma], axis=2)
    recurrence_matrix = librosa.segment.recurrence_matrix(chroma_stack[:,:,0])
    
    # 6. 시각화
    plt.figure(figsize=(12, 8))
    
    plt.subplot(3, 1, 1)
    librosa.display.waveshow(y, sr=sr, alpha=0.8)
    plt.title('Waveform')
    plt.vlines(beats, -1, 1, color='red', alpha=0.6, linestyle='--', label='Beats')
    plt.legend()
    
    plt.subplot(3, 1, 2)
    librosa.display.specshow(chroma, y_axis='chroma', x_axis='time', sr=sr)
    plt.title('Chromagram')
    plt.colorbar()
    
    plt.subplot(3, 1, 3)
    librosa.display.specshow(mfcc, x_axis='time', sr=sr)
    plt.title('MFCC')
    plt.colorbar()
    
    plt.tight_layout()
    plt.show()
    
    # 7. 분석 결과 구조화
    analysis_result = {
        "bpm": float(tempo),
        "genre": predicted_genre,
        "confidence": 0.75,  # 간단한 분석이므로 낮은 신뢰도
        "features": {
            "brightness": float(brightness),
            "rhythmic_regularity": float(rhythmic_regularity),
            "duration": len(y) / sr,
            "beats_count": len(beats)
        },
        "technical": {
            "sample_rate": sr,
            "channels": 1,
            "format": "librosa_analyzed"
        }
    }
    
    # 8. GCP 업로드 (선택적)
    if upload_to_gcp:
        file_name = upload_analysis_result(
            "music-analysis",
            {"file_path": file_path, "method": "librosa"},
            analysis_result,
            processing_time_ms=2000  # 대략적인 처리 시간
        )
        print(f"☁️ GCP 업로드 완료: {file_name}")
    
    return analysis_result

# GPU 가속 오디오 처리 (Colab Pro 전용)
def gpu_accelerated_analysis(file_path):
    """
    GPU를 활용한 대용량 오디오 분석
    RENDER에서는 불가능한 GPU 리소스 활용
    """
    try:
        import torch
        import torchaudio
        
        if torch.cuda.is_available():
            device = torch.device('cuda')
            print(f"🚀 GPU 가속 활성화: {torch.cuda.get_device_name()}")
            
            # GPU에서 오디오 로드
            waveform, sample_rate = torchaudio.load(file_path)
            waveform = waveform.to(device)
            
            # GPU 기반 스펙트로그램 계산
            spectrogram = torchaudio.transforms.Spectrogram().to(device)
            spec = spectrogram(waveform)
            
            print(f"✅ GPU 분석 완료: {spec.shape}")
            return spec.cpu().numpy()
        else:
            print("⚠️ GPU 사용 불가, CPU로 대체")
            return analyze_audio_file(file_path, upload_to_gcp=False)
            
    except ImportError:
        print("⚠️ PyTorch 미설치, librosa로 대체")
        return analyze_audio_file(file_path, upload_to_gcp=False)

# 사용 예시 (샘플 오디오 파일이 있을 때)
print("🎵 음악 분석 도구 준비 완료!")
print("사용법:")
print("1. result = analyze_audio_file('your_music.wav')")
print("2. gpu_result = gpu_accelerated_analysis('your_music.wav')  # GPU 가속")